In [ ]:
import sys
import platform
from datetime import datetime, timedelta
import tkinter as tk
from tkinter import messagebox

# Optional: better beep on Windows using built-in winsound
WIN = platform.system() == "Windows"
if WIN:
    import winsound

def now_local():
    return datetime.now()

def parse_time(hhmm: str, ampm: str) -> datetime | None:
    try:
        hh, mm = hhmm.strip().split(":")
        h = int(hh)
        m = int(mm)
        if ampm == "AM":
            h = 0 if h == 12 else h
        else: # PM
            h = 12 if h == 12 else h + 12
        today = now_local().date()
        return datetime.combine(today, datetime.min.time()).replace(hour=h, minute=m, second=0, microsecond=0)
    except Exception:
        return None

class AlarmClock(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Alarm Clock")
        self.geometry("360x230")
        self.resizable(False, False)

        # UI
        self.time_label = tk.Label(self, text="", font=("Helvetica", 28, "bold"))
        self.time_label.pack(pady=10)

        row = tk.Frame(self)
        row.pack(pady=5)

        tk.Label(row, text="Alarm time (HH:MM)").grid(row=0, column=0, padx=6, sticky="e")
        self.time_entry = tk.Entry(row, width=7, justify="center")
        self.time_entry.insert(0, "07:00")
        self.time_entry.grid(row=0, column=1)

        tk.Label(row, text="AM/PM").grid(row=0, column=2, padx=6)
        self.ampm = tk.StringVar(value="AM")
        tk.OptionMenu(row, self.ampm, "AM", "PM").grid(row=0, column=3)

        self.repeat = tk.BooleanVar(value=True)
        tk.Checkbutton(self, text="Repeat daily", variable=self.repeat).pack()

        btns = tk.Frame(self)
        btns.pack(pady=8)
        tk.Button(btns, text="Set Alarm", width=12, command=self.set_alarm).grid(row=0, column=0, padx=6)
        tk.Button(btns, text="Stop", width=12, command=self.stop_alarm).grid(row=0, column=1, padx=6)
        tk.Button(btns, text="Snooze 5 min", width=12, command=self.snooze).grid(row=0, column=2, padx=6)

        self.status = tk.Label(self, text="No alarm set.", fg="gray")
        self.status.pack(pady=4)

        # State
        self.next_alarm_dt: datetime | None = None
        self.ringing = False

        # Start clock updates
        self.update_clock()

    def update_clock(self):
        self.time_label.config(text=now_local().strftime("%I:%M:%S %p"))
        self.check_alarm()
        self.after(250, self.update_clock)

    def set_alarm(self):
        t = parse_time(self.time_entry.get(), self.ampm.get())
        if not t:
            messagebox.showerror("Invalid time", "Please enter time as HH:MM (e.g., 07:30)")
            return
        now = now_local()
        # schedule today or tomorrow
        if t <= now:
            t = t + timedelta(days=1)
        self.next_alarm_dt = t
        self.ringing = False
        self.status.config(text=f"Alarm set for {self.next_alarm_dt.strftime('%I:%M %p on %b %d')} "
                                f"({'daily' if self.repeat.get() else 'once'})", fg="green")

    def stop_alarm(self):
        if self.ringing:
            self.ringing = False
        # If it was a repeating alarm, keep the next day; if one-shot, clear
        if self.next_alarm_dt and not self.repeat.get():
            self.next_alarm_dt = None
            self.status.config(text="Alarm cleared.", fg="gray")
        elif self.next_alarm_dt and self.repeat.get():
            # move to next day silently
            self.next_alarm_dt = self.next_alarm_dt + timedelta(days=1)
            self.status.config(text=f"Next alarm: {self.next_alarm_dt.strftime('%I:%M %p on %b %d')} (daily)", fg="green")

    def snooze(self):
        if not self.next_alarm_dt and not self.ringing:
            messagebox.showinfo("Snooze", "No active alarm to snooze.")
            return
        self.ringing = False
        self.next_alarm_dt = now_local() + timedelta(minutes=5)
        self.status.config(text=f"Snoozed to {self.next_alarm_dt.strftime('%I:%M %p')}", fg="orange")

    def beep(self):
        # Try a nicer Windows beep; otherwise use Tk bell as a fallback
        if WIN:
            try:
                # frequency, duration(ms)
                winsound.Beep(880, 300)
                winsound.Beep(988, 300)
                winsound.Beep(1047, 400)
            except RuntimeError:
                self.bell()
        else:
            # cross-platform Tk bell (may be muted on some systems)
            self.bell()

    def ring(self):
        if not self.ringing:
            self.ringing = True
            messagebox.showinfo("Alarm", "⏰ Time's up!")
        self.beep()

    def check_alarm(self):
        if not self.next_alarm_dt:
            return
        now = now_local()
        # trigger when time has passed or is within 1 second
        if not self.ringing and now >= self.next_alarm_dt:
            self.ring()
            # schedule next occurrence if repeating
            if self.repeat.get():
                self.next_alarm_dt = self.next_alarm_dt + timedelta(days=1)
                self.status.config(text=f"Next alarm: {self.next_alarm_dt.strftime('%I:%M %p on %b %d')} (daily)", fg="green")
            else:
                # one-shot: keep ringing until Stop or Snooze
                self.status.config(text="Alarm ringing! Press Stop or Snooze.", fg="red")

if __name__ == "__main__":
    try:
        app = AlarmClock()
        app.mainloop()
    except KeyboardInterrupt:
        sys.exit(0)